# Embeddings Neuronales

En este notebook, se utiliza la librería [Word2vec](https://en.wikipedia.org/wiki/Word2vec) como modelo de generación de embeddings de palabras, luego se entrena un modelo k-means para obtener clusters, y se marca cada comentario con un determinado número de cluster.

### Importación de librerías requeridas

In [1]:
import os
import pickle
import warnings

import gensim.corpora as corpora
from gensim.models import Word2Vec

from collections import Counter
import numpy as np

import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances, silhouette_samples, silhouette_score

from clustering_utils import vectorize, mbkmeans_clusters

warnings.filterwarnings("ignore", category=DeprecationWarning)

np.random.seed(42)

%matplotlib inline


### Definición de variables globales

In [2]:
TEXT_FILE_READ = 'docs/preprocessing_reddit_data.csv'
TEXT_SAVE_FILE = 'docs/reddit_data_word2vec.csv'
PICKLE_KMEANS = 'docs/models/word2vec_kmeans.model'
TEST_CLUSTER_PATH = 'docs/test/w2v_comments_per_cluster/'
SAVE_TEST_CSV=False

n_clusters = 120

### Lectura de los comentarios de Reddit

Los comentarios fueron previamente preprocesados (Ver en TODO).

In [3]:
df = pd.read_csv(TEXT_FILE_READ)
df['lemma_tokens'] = df['lemma_tokens'].apply(eval)

### Vocabulario

In [4]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemma_tokens'])

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)

# Creating a corpus object
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]


In [5]:
processed_corpus = df['lemma_tokens']


### Entrenamiento del modelo Word2Vec

In [6]:
model = Word2Vec(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=1)
model.train(processed_corpus, total_examples=len(processed_corpus), epochs=100)
model.save("docs/models/word2vec.model")

In [7]:
word_vecs = []
vocabulary = list(model.wv.key_to_index)

for key in model.wv.key_to_index:
    word_vecs.append(model.wv[key])

### Generación de vectores desde documentos

In [19]:
vectorized_docs = vectorize(processed_corpus, model=model)
len(vectorized_docs), len(vectorized_docs[0])

(27791, 100)

### Generación de clusters

In [10]:
clustering, cluster_labels = mbkmeans_clusters(
    X=vectorized_docs,
    k=n_clusters,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": df["body"].values,
    "tokens": [" ".join(text) for text in processed_corpus],
    "cluster": cluster_labels
})

with open(PICKLE_KMEANS, 'wb') as f:
    pickle.dump(clustering, f)

For n_clusters = 120
Silhouette coefficient: 0.02
Inertia:863175.8595201003
Silhouette values:
    Cluster 67: Size:70 | Avg:0.16 | Min:0.01 | Max: 0.38
    Cluster 45: Size:338 | Avg:0.15 | Min:-0.01 | Max: 0.32
    Cluster 25: Size:57 | Avg:0.15 | Min:0.02 | Max: 0.37
    Cluster 77: Size:29 | Avg:0.14 | Min:-0.07 | Max: 0.34
    Cluster 75: Size:52 | Avg:0.14 | Min:-0.03 | Max: 0.33
    Cluster 73: Size:100 | Avg:0.13 | Min:-0.06 | Max: 0.33
    Cluster 53: Size:143 | Avg:0.12 | Min:-0.02 | Max: 0.33
    Cluster 66: Size:120 | Avg:0.12 | Min:-0.03 | Max: 0.32
    Cluster 27: Size:191 | Avg:0.12 | Min:-0.11 | Max: 0.35
    Cluster 118: Size:44 | Avg:0.12 | Min:-0.13 | Max: 0.34
    Cluster 4: Size:41 | Avg:0.12 | Min:-0.08 | Max: 0.32
    Cluster 113: Size:1810 | Avg:0.11 | Min:0.02 | Max: 0.22
    Cluster 64: Size:117 | Avg:0.11 | Min:-0.05 | Max: 0.32
    Cluster 80: Size:147 | Avg:0.11 | Min:-0.05 | Max: 0.31
    Cluster 104: Size:219 | Avg:0.11 | Min:-0.17 | Max: 0.37
    Cluster

### *Top terms* por cluster (basado en los centroides de los clusters)

In [11]:
print("Most representative terms per cluster (based on centroids):")
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")



Most representative terms per cluster (based on centroids):
Cluster 0: hacer dislocar desuscribite ss vtv 
Cluster 1: ss suggest childrir opinión changes 
Cluster 2: empezar él propusar denno panquequear 
Cluster 3: amigo empija amateur amiga chonga 
Cluster 4: ️ ♂ ↩ ↪ ‍ 
Cluster 5: gente mayoría riesgo dirijar nefast 
Cluster 6: vida felicidad resolvemos chotado congruente 
Cluster 7: país población europeo pobreza encendido 
Cluster 8: precio generador explayartar inflacion suba 
Cluster 9: temaso suggest maal negro_con_dedo_en_la_frente.jpg obliguen 
Cluster 10: yo pensar ss suggest childrir 
Cluster 11: sacar salvé mandé propelar pakistán 
Cluster 12: animación atardecer floxie10 muetra memorio 
Cluster 13: comprar vender llenalo cotillón vendo 
Cluster 14: estofado caloría anchoar acompañamiento gin 
Cluster 15: andar canaaaaas bici cambialo gif](giphy|3o7adangmejzswe8cu|downsized 
Cluster 16: seguir medida childrir pijar puesto 
Cluster 17: x200b mirandar mierdo mierda childrir 


### *Top terms* por cluster (basado en las palabras más frecuentes)

In [12]:
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_frequent = Counter(" ".join(df_clusters.query(f"cluster == {i}")["tokens"]).split()).most_common(5)
    for t in most_frequent:
        tokens_per_cluster += f"{t[0]}({str(t[1])}) "
    print(f"Cluster {i}: {tokens_per_cluster}")


Cluster 0: hacer(286) él(76) querer(16) decir(11) paja(10) 
Cluster 1: él(105) malo(91) preguntar(71) serio(51) entender(46) 
Cluster 2: empezar(9) él(9) esperen(1) agarro(1) pochoc(1) 
Cluster 3: amigo(158) él(7) pasar(6) decir(5) mirar(4) 
Cluster 4: ️(49) ✌(24) ♂(17) ‍(13) 🤦(10) 
Cluster 5: gente(290) vivir(16) él(13) ver(11) creer(10) 
Cluster 6: vida(219) él(13) vivir(11) real(8) querer(8) 
Cluster 7: país(315) gente(19) mundo(17) él(15) argentino(15) 
Cluster 8: precio(193) inflación(30) bajar(22) controlar(22) control(15) 
Cluster 9: noche(54) mañana(52) dormir(46) él(46) viernes(41) 
Cluster 10: yo(159) pensar(119) él(63) decir(18) ir(17) 
Cluster 11: sacar(140) él(29) foto(10) gente(5) plata(5) 
Cluster 12: video(96) serie(47) ver(42) historia(29) él(20) 
Cluster 13: comprar(135) vender(105) él(32) precio(16) barato(14) 
Cluster 14: rico(36) carne(33) dulce(28) grasa(28) mate(26) 
Cluster 15: andar(92) caer(59) bici(10) culo(7) él(6) 
Cluster 16: seguir(182) él(20) pasar(15) a

### Recupere los documentos más representativos (basados en los centroides de los clústeres) para un cluster en particular

In [45]:
test_cluster = 17
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:20]:
    print( df["body"].values[d])
    print("-------------")

Por qué no las dos ? 0w0
-------------
Momentito! Nadie que programe podria decir que la ayuda con la ira cuando jamas compila de 1 y siempre hay bugs, deadlines de mierda.. Matenla muchachos!!!!
-------------
Como no me avive de la santilleta
-------------
Por lo menos en Oaxaca hacen mezcal
-------------
Era un chicle bazooka en el 2001
-------------
Es una nueva asesora de Fabiola?
-------------
No puede ser que nadie haya dicho el Suchard, ese "alfafor" es lo mas!!!
-------------
esta bueno el.sistema meritocratico que usan.
-------------
Creo que bonitas.com no existe pero IAMC si. Muchas gracias!
-------------
Que cervecería era? De dónde? Así los birreros sabemos
-------------
Como todo autista que se precie
-------------
"eL caPiTalzMo eZ Y lO imVenTo uSA!!"
-------------
Ticketek dice, voy a ver si puedo consultarles
-------------
no somo potensia porque no queremo
-------------
a pero la Asamblea de 1813
-------------
pero bien que se te frunce el * en Salsipuedes...
--------

In [14]:
#solo test
#print(len(vectorized_docs))
#print(vectorized_docs[0])

test_v = vectorize([['defender', 'peso', 'siente', 'corazón', 'compro', 'pesos', 'tasa', 'fijo', 'año']], model=model)
prediction = clustering.predict(test_v)
print(prediction)

ver = "['defender', 'peso', 'siente', 'corazón', 'compro', 'pesos', 'tasa', 'fijo', 'año']"
ver = "tapastir baño tirar balde aguo"
test_v = vectorize([ver], model=model)
prediction = clustering.predict(test_v)
print(prediction)


str2 = ver.split(" ")
print(str2)

[23]
[113]
['tapastir', 'baño', 'tirar', 'balde', 'aguo']


In [15]:
reddit = pd.read_csv(TEXT_FILE_READ)

def get_cluster(row):
    test_v = vectorize([str(row).split(" ")], model=model)
    return clustering.predict(test_v)

reddit['cluster'] = reddit.apply(lambda row: get_cluster(row['body_preprocessing']) , axis = 1) 


In [16]:
# Show
df.head(10)

,score,id,flair,comms_num,body,comment_parent_id,is_replay,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,lemma_tokens,body_preprocessing,cluster
0,1,hfw14mt,Discusion🧐,1,todo para decir que tapaste el baño. tira un b...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tapastir', 'baño', 'tirar', 'balde', 'aguo']",tapastir baño tirar balde aguo,[68]
1,1,hfw41eh,Discusion🧐,0,"sopapa primero master, si hay tapón te vas a t...",hfw14mt,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapa', 'master', 'tapón', 'va', 'teñir', '...",sopapa master tapón va teñir medio,[66]
2,1,hfw1ao2,Discusion🧐,0,"Usas la sopapa, o tiras agua caliente con un b...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapo', 'tira', 'agua', 'caliente', 'balde']",sopapo tira agua caliente balde,[48]
3,1,hfw3jof,Discusion🧐,2,Lo que he probado que siempre me dio resultado...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['probado', 'resultado', 'sellar', 'boca', 'in...",probado resultado sellar boca inodoro tirar ca...,[47]
4,1,hfw6v4i,Discusion🧐,0,Estas cobrando por dar mantenimiento y no sabe...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['cobrar', 'mantenimiento', 'carajo', 'kjjjjjj...",cobrar mantenimiento carajo kjjjjjjjjj vivirio...,[76]
5,1,hfw26iv,Discusion🧐,0,"Si tenes algo con punta, metelo y hace un poco...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tén', 'punto', 'metelo', 'fuerza', 'romper',...",tén punto metelo fuerza romper tapo baño tirar...,[48]
6,1,hfw2gof,Discusion🧐,1,"Con una manguera para regar el jardín, si tene...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['regar', 'jardín', 'tén', 'pod', 'probar']",regar jardín tén pod probar,[84]
7,1,hfw5s13,Discusion🧐,0,"despues regas el jardin y se lava sola, solo q...",hfw2gof,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['rega', 'jardin', 'lava', 'tenés', 'lavarte',...",rega jardin lava tenés lavarte mano pulgar chorro,[34]
8,1,hfw3air,Discusion🧐,0,La respuesta real es que se venden unos caños ...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['respuesta', 'real', 'vender', 'caño', 'alamb...",respuesta real vender caño alambrado decir cañ...,[114]
9,7,hfvxa6w,Discusion🧐,3,Mi alfajor favorito es el Havana,q443eo,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['alfajor', 'favorito', 'haván']",alfajor favorito haván,[14]


In [17]:
df.to_csv(TEXT_SAVE_FILE, index=False)

In [18]:
if SAVE_TEST_CSV:
    os.makedirs(TEST_CLUSTER_PATH,exist_ok=True)

    for i in range(n_clusters):
        df[(df["cluster"] == i)][['flair', 'body']].to_csv(TEST_CLUSTER_PATH + str(i) + '.csv')
